In [48]:
import json
from climdata.extremes.indices import extreme_index 
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          13.246667038198012,
          52.891982026993958
        ],
        "type": "Point"
      }
    }
  ]
}


import climdata
clim = climdata.extract_data(
    overrides=[
            "dataset=mswx",
            "variables=['tas', 'tasmin', 'sfcWind']",
            "data_dir=/beegfs/muduchuru/data", #optional 
            f"time_range.start_date=1989-01-01",
            f"time_range.end_date=2014-12-31",
            f"aoi='{json.dumps(geojson)}'",
            # "index=SPI1",
            "dsinfo.mswx.params.google_service_account=./.climdata_conf/service.json",
    ]
,
save_to_file=False
)

../../usecase/conf
False


ValueError: No dataset loaded. Call `load()` first.

In [47]:
import os
os.environ["IMPUTEGAP_ENV_PATH"] = "/beegfs/muduchuru/pkgs_fnl/climdata/usecase/brits_config.toml"

import numpy as np
from imputegap.recovery.imputation import Imputation
from imputegap.recovery.manager import TimeSeries
import toml
import imputegap
import os

# Extract multiple variables for a single station
custom_data = clim[1][["tas",'tasmin']].resample(time="1D").mean().to_array().mean("station_id")

ts = TimeSeries()
ts.data = custom_data.copy().values
ts.n_series, ts.seq_len = ts.data.shape  # (4, time)

# Optional: normalize
ts.normalize(normalizer="z_score")

# Contaminate the data (MCAR)
ts_m = ts.Contamination.mcar(ts.data, rate_dataset=0.05, rate_series=0.05)
# ts_m = np.asarray(ts_m)  # ensure NumPy array


> logs: normalization (z_score) of the data - runtime: 0.0002 seconds

(CONT) missigness pattern: MCAR
	selected series: 2
	percentage of contaminated series: 5.0%
	rate of missing data per series: 5.0%
	block size: 10
	security offset: [0-651]
	seed value: 42


KeyboardInterrupt: 

In [ ]:
import os
import tempfile
import numpy as np
import pandas as pd
import hvplot.pandas
import xarray as xr

# --- 5️⃣ Run BRITS ---
imputer = Imputation.MatrixCompletion.CDRec(ts_m)
imputer.impute()  # will use 5 epochs

# --- 6️⃣ Recovered data ---
recovered = imputer.recov_data

# --- 7️⃣ Interactive plotting with hvPlot ---
df = pd.DataFrame({
    "original": ts.data[0],
    # Only show original values where missing
    "missing": np.where(np.isnan(ts_m[0]), ts.data[0], np.nan),
    # Only show recovered values where missing
    "recovered": np.where(np.isnan(ts_m[0]), recovered[0], np.nan)
})

df.hvplot(title="MPIN Imputation", height=300)


ERROR: Rank choosen to high for the number of series: 3 >= 2.
	Rank reduced to 2.

(IMPUTATION) CDRec
	Matrix: 2, 9496
	truncation rank: 2
	epsilon: 1e-06
	iterations: 100

> logs: imputation cdrec - Execution Time: 0.3654 seconds.


:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [32]:
ts.plot(input_data=ts.data, incomp_data=ts_m, recov_data=imputer.recov_data, nbr_series=9, subplot=True, algorithm=imputer.algorithm, save_path="./imputegap_assets/imputation")


plots saved in: ./imputegap_assets/imputation/25_11_26_13_36_16_mpin_plot.jpg


'./imputegap_assets/imputation/25_11_26_13_36_16_mpin_plot.jpg'

In [35]:
imputer.score(ts.data, imputer.recov_data)
ts.print_results(imputer.metrics)


Results :
RMSE                 = 0.29017749830807504
MAE                  = 0.23852259275095283
MI                   = 1.0073774172207155
CORRELATION          = 0.9553585676700678


In [36]:
ts.data

array([[-0.6118232 , -0.67718905, -1.2001158 , ..., -1.87992058,
        -1.44850601, -0.93865242],
       [-0.8332815 , -0.28144307, -0.98242702, ..., -1.93695836,
        -1.38511993, -0.62447777]])

In [ ]:
df = pd.DataFrame({
    "original": ts.data[0],
    "missing": np.where(np.isnan(ts_m[0]), np.nan, ts.data[0]),
    "recovered": recovered[0]
})
df.hvplot(title="BRITS Imputation", height=300)

470

In [ ]:
import hvplot.xarray
(ts.data.to_pandas().T.hvplot() + imputer.recov_data.hvplot() + ts_m.hvplot())

Init signature: Imputation.DeepLearning.BRITS(incomp_data)
Docstring:     
BRITS class to impute missing values using Bidirectional Recurrent Imputation for Time Series

Methods
-------
impute(self, user_def=True, params=None):
    Perform imputation using the BRITS algorithm.
Init docstring:
Initialize the BaseImputer with an infected time series matrix.

Parameters
----------
incomp_data : numpy.ndarray
    Matrix used during the imputation of the time series.
File:           ~/miniforge3/envs/sdba/lib/python3.10/site-packages/imputegap/recovery/imputation.py
Type:           type
Subclasses:     